# Dev 09 - Pulse Compression

In [ ]:
import os
import time
import numpy as np
import scipy as sp
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('ggplot')

import cspec
import toshi

zmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.zmap()[:, :3])
vmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', cspec.colormap.vmap()[:, :3])

In [ ]:
waveform = toshi.tx_waveform()
wtc_loc = toshi.wtc_loc_from_csv()

turbines = np.array([[pt[2], pt[1]] for pt in wtc_loc], dtype=np.single)

In [ ]:
fs = 2e6
t = np.arange(len(waveform)) / fs * 1e6

plt.figure(figsize=(9, 2.5), dpi=144)
plt.plot(t, waveform.real, linewidth=0.75)
plt.plot(t, waveform.imag, linewidth=0.75)
plt.xlim((0, 40))
plt.xlabel('Time (us)')

In [ ]:
pc_nfft = 1024
wf = sp.fft.fft(waveform / np.sqrt(np.sum(np.abs(waveform) ** 2)), n=pc_nfft)

In [ ]:
# file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')
# file = os.path.expanduser('~/Downloads/20200615_010423.194421-8C-02.iqData.XXXX.AKITA.dat')
# file = os.path.expanduser('~/Downloads/toshiba/AKITA_IQ_TO_OU/IQdata/2021.01.28/12/20210128_124100.876775-CD-68.iqData.XXXX.AKITA.dat')
file = os.path.expanduser('~/Downloads/toshiba/AKITA_IQ_TO_OU/IQdata/2021.01.28/12/20210128_124008.095007-CA-68.iqData.XXXX.AKITA.dat')

filesize = os.path.getsize(file)
print('{} - {:,.0f} B'.format(file, filesize))

In [ ]:
s = time.time()
all_ray_pulses, all_cpi_headers = toshi.read(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

In [ ]:
# Go through the pulses for azimuth
az = np.zeros(len(all_ray_pulses), dtype=np.single)
for k, pulses in enumerate(all_ray_pulses):
    az[k] = pulses[0].azimuth
    
# Choose ray 2 to whatever that completes the 360-deg coverage
n = np.argmin(np.abs(az[3:] - az[2])) + 1
az = az[2:n+2]
ray_pulses = all_ray_pulses[2:n+2]
cpi_headers = all_cpi_headers[2:n+2]

# Dimensions
naz = len(ray_pulses)
ngate_long = ray_pulses[0][0].cpi_header.num_range_long_hi
ngate_short_hi = ray_pulses[0][0].cpi_header.num_range_short_hi
ngate_short_lo = ray_pulses[0][0].cpi_header.num_range_short_lo
ngate = ngate_long + ngate_short_hi + ngate_short_lo

# Elevation assumed to be flat from the very first pulse
scan_el = ray_pulses[0][0].elevation
scan_time = time.strptime(os.path.basename(file)[:15], '%Y%m%d_%H%M%S')

# Sampling code from the CPI header
fs = 1.0e6 * (1 << cpi_headers[0].fs_code)
dr = 3.0e8 / fs / 2
r = 1.0e-3 * np.arange(0, ngate, dtype=np.single) * dr + 0.5 * dr

print('fs = {:,.0f} Hz    n = {} -> 2 ... {} ({})'.format(fs, n, n+2, naz))

In [ ]:
# Noise estimate, try azimuth 0, around 20-25 km
# ia = np.argmin(np.abs(a[3:] - 0.0)) + 1
# ir, er = np.argmin(np.abs(r - 20.0)), np.argmin(np.abs(r - 25.0))
# samples = np.zeros((len(ray_pulses[ia]), er-ir), dtype=np.csingle)

# Gather the samples. Ignore phase code since we are only interested in amplitude
# for k, pulse in enumerate(ray_pulses[ia]):
#     samples[k, :] = pulse.h_long_hi[ir:er]
# noise = np.mean(np.abs(samples)) ** 2

# Hard code noise estimate to be about 24 (eye ball)
noise = 24
print('Using noise estimate in 16-bit ADU: {:.4f}'.format(noise))

In [ ]:
s = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)

nfft = 256
use_window = True
use_pulse_pair = False

# Go through the pulses
a = np.zeros(len(ray_pulses), dtype=np.single)
for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    a[k] = pulses[0].azimuth
    npulse = len(pulses)

    # Decode the long pulse, then compress using wf
    p = np.zeros((npulse, ngate_long), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, :] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
    pf = sp.fft.fft(p, n=pc_nfft, axis=1)
    pc = sp.fft.ifft(pf * wf, n=pc_nfft, axis=1)

    # Gather the short pulses and the compressed long pulses
    p = np.zeros((npulse, ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        c = np.exp(-1j * pulse.phase_h_short)                              # Phase code
        p[j, :ngate_long] = pc[j, :ngate_long]                             # Long only
        p[j, :ngate_short_hi] = pulse.h_short_hi * c                       # Short hi
        p[j, :ngate_short_lo] = pulse.h_short_lo * c                       # Short lo        
    p = p - np.mean(p, axis=0)                                             # Remove DC if desired

    if use_window:
        w = scipy.signal.get_window('blackmanharris', len(p))
    else:
        w = np.ones((p.shape[0],))
    w /= np.sqrt(np.sum(w ** 2)) / np.sqrt(p.shape[0])                     # Normalize to non-windowed gain
    ww = np.repeat(np.expand_dims(w, axis=1), ngate, axis=1)               # Make same shape
    p *= ww                                                                # Windowing

    if use_pulse_pair:
        pp = p[1:, :] * np.conj(p[:-1, :])                                 # x(n) * x'(n-1)
        s[k, :] = np.mean(np.abs(p) ** 2, axis=0)                          # s(n) = E[x(n) * x'(n)]
        v[k, :] = np.angle(np.sum(pp, axis=0))                             # r(1) = E[x(n) * x'(n-1)]
    else:
        spec = np.fft.fft(p, nfft, axis=0)                                 # FFT
        s[k, :] = np.mean(np.abs(spec) ** 2, axis=0) / p.shape[0]          # Periodogram
        v[k, :] = np.angle(np.fft.ifft(spec * np.conj(spec), axis=0)[1])   # IFT -> ACF[1]

# Signal
s -= noise
s[s <= 0] = 1.0e-6                                    # Avoid log(0)
snr = 10 * np.log10(s / noise)                        # Signal-to-noise ratio in dB
z = 10 * np.log10(s * (r + 0.5e-3 * dr) ** 2) - 40    # Estimated ZCal = -40 
z[:, :ngate_short_hi] += 16                           # ~18-dB on the short waveform, perhaps?

# Thresholding at SNR = 0 dB
m = snr < -3
z[m] = np.nan
v[m] = np.nan

In [ ]:
dr

In [ ]:
# Edge of range cells
wid_a = np.mean(sorted(np.diff(az))[int(0.3 * len(az)):int(0.6 * len(az))])
end_a = az[-1] + wid_a
if end_a >= 360.0:
    end_a -= 360.0
ae = np.append(az, end_a)
re = 1.0e-3 * np.arange(0, ngate + 1, dtype=np.single) * dr

In [ ]:
np.outer(np.arange(1, 10), np.sin(ring_a)).shape

In [ ]:
ring_x.shape

In [ ]:
# Radar cell locations
ce = np.cos(np.deg2rad(scan_el))
rr, aa = np.meshgrid(re, np.deg2rad(ae))
xx = rr * ce * np.sin(aa)
yy = rr * ce * np.cos(aa)

# ring_r = 7.0
# ring_a = np.arange(0, 360, 2) / 180 * np.pi
# ring_x = ring_r * np.sin(ring_a)
# ring_y = ring_r * np.cos(ring_a)
ring_r = np.arange(1, 11)
ring_a = np.arange(0, 361, 2) / 180 * np.pi
ring_x = np.outer(np.sin(ring_a), ring_r)
ring_y = np.outer(np.cos(ring_a), ring_r)

# Turbine locations
a_turb = turbines[:, 0] / 180.0 * np.pi
r_turb = turbines[:, 1]
x_turb = r_turb * np.sin(a_turb)
y_turb = r_turb * np.cos(a_turb)

# Various domain to choose from
# xlim, ylim = (-3, 2), (2, 6.5)
# xlim, ylim = (-7, 6.5), (-6, 7)
# xlim, ylim = (-13, 5), (3, 18)
# xlim, ylim = (-25, 10), (0, 35)
# xlim, ylim = (-20, 20), (-20, 20)
xlim, ylim = (-8.4, 8.4), (-8.4, 8.4)

# Reflectivity plot
plt.figure(figsize=(5, 4), dpi=216)
plt.pcolormesh(xx[:, :501], yy[:, :501], z[:, :500], cmap=zmap)
# plt.plot(x_turb, y_turb, 'xk', markersize=4, linewidth=0.33)
plt.plot(ring_x, ring_y, 'k:', linewidth=0.5)
for i in range(ring_x.shape[1]):
    plt.text(ring_x[67, i], ring_y[67, i], '{} km'.format(i+1), fontsize=6, ha='center', va='center', rotation=45)
# plt.plot(ring_x[-1, :], ring_y[-1, :], 'k:')
plt.clim((-32, 96))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)

# Velocity plot
plt.figure(figsize=(5, 4), dpi=216)
plt.pcolormesh(xx[::2, :501], yy[::2, :501], v[::2, :500], cmap=vmap)
# plt.plot(x_turb, y_turb, 'xk', markersize=4, linewidth=0.33)
plt.clim((-5, 5))
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.grid(linewidth=0.25)
plt.title(os.path.basename(file), fontsize=8)

plt.show()

In [ ]:
ray_pulses[0][0].cpi_header.align_num_range_long_hi, ray_pulses[0][0].cpi_header.align_num_range_long_lo, ray_pulses[0][0].cpi_header.align_num_range_short_hi, ray_pulses[0][0].cpi_header.align_num_range_short_lo

In [ ]:
ray_pulses[0][0].cpi_header.num_range_long_hi, ray_pulses[0][0].cpi_header.num_range_long_lo, ray_pulses[0][0].cpi_header.num_range_short_hi, ray_pulses[0][0].cpi_header.num_range_short_lo

In [ ]:
# ray_pulses[0][0].cpi_header.num_range_long_hi, ray_pulses[0][0].cpi_header.num_range_long_lo, ray_pulses[0][0].cpi_header.num_range_short_hi, ray_pulses[0][0].cpi_header.num_range_short_lo